REQUIRMENTS

In [ ]:
%pip install tflearn

In [ ]:
%pip uninstall tensorflow

In [ ]:
%pip install tensorflow

In [ ]:
%pip install nltk

In [ ]:
#nltk.download()

COGNITIVE DISTORTION DETECTION MODEL

In [2]:
%pip install sklearn

In [4]:
%pip install scikit-learn

     ---------------------------------------- 8.2/8.2 MB 18.8 MB/s eta 0:00:00
     ---------------------------------------- 40.1/40.1 MB 3.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install imbalanced-learn

In [ ]:
%pip install pandas

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.pipeline import Pipeline, make_pipeline
from imblearn.over_sampling import SMOTE
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
import pandas as pd

# Loading The Dataset 
df = pd.read_csv('./Dataset_CD - Full 17.tsv', sep='\t', header=0)
df = df.drop(df.columns[[2, 3, 4]], axis=1)  # df.columns is zero-based pd.Index
#df.shape

# Divide data between training and test data
X = df['Phrase']
y = df['Cognitive Distortion']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, train_size = .8)

textclassifier =Pipeline([
   ('vect', CountVectorizer()),
   ('tfidf', TfidfTransformer()),
   ('smote', SMOTE(random_state=0)),
   ('lg', LogisticRegression(multi_class='multinomial', random_state=0))
])

params = {'smote__k_neighbors': [2,3,4,5,6,7,8,9,10, ],
          'lg__penalty': ['l1', 'l2', 'elasticnet', 'none' ]}
 
logistic_regression_grid = GridSearchCV(estimator=textclassifier, param_grid=params, n_jobs=10, cv=10, verbose=5)
logistic_regression_grid.fit(X_train, y_train)

print('Train Accuracy : %.3f'%logistic_regression_grid.best_estimator_.score(X_train, y_train))
print('Test Accuracy : %.3f'%logistic_regression_grid.best_estimator_.score(X_test, y_test))
print('Best Accuracy Through Grid Search : %.3f'%logistic_regression_grid.best_score_)
print('Best Parameters : ',logistic_regression_grid.best_params_)

Fitting 10 folds for each of 36 candidates, totalling 360 fits


C:\Users\Usuario\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
180 fits failed out of a total of 360.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
90 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Usuario\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Usuario\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\lo

Train Accuracy : 0.998
Test Accuracy : 0.716
Best Accuracy Through Grid Search : 0.700
Best Parameters :  {'lg__penalty': 'none', 'smote__k_neighbors': 9}


In [3]:
import pickle

pickle.dump(logistic_regression_grid, open('cognitive_distortion_detector_model.pkl', 'wb'))


RULE BASED MODEL

In [5]:
# Importing modules
import re
from nltk.corpus import wordnet

import itertools

# Building a list of Keywords
list_words=[
    'hello','bye',
    'suicide','kill','die','want','feel',
    'stupid','dumb','asshole',
    'think', 'was','thought','thinking'
    ]
list_syn={}
for word in list_words:
    synonyms=[]
    for syn in wordnet.synsets(word):
        for lem in syn.lemmas():
            # Remove any special characters from synonym strings
            lem_name = re.sub('[^a-zA-Z0-9 \n\.]', ' ', lem.name())
            synonyms.append(lem_name)
    list_syn[word]=set(synonyms)
print (list_syn)

# Building dictionary of Intents & Keywords
keywords={}
keywords_dict={}
# Defining a new key in the keywords dictionary
keywords['greetings']=[]
# Populating the values in the keywords dictionary with synonyms of keywords formatted with RegEx metacharacters 
for synonym in list(list_syn['hello']):
    keywords['greetings'].append('.*\\b'+synonym+'\\b.*')

# Defining a new key in the keywords dictionary
keywords['goodbyes']=[]
# Populating the values in the keywords dictionary with synonyms of keywords formatted with RegEx metacharacters 
for synonym in list(list_syn['bye']):
    keywords['goodbyes'].append('.*\\b'+synonym+'\\b.*')

# Defining a new key in the keywords dictionary
keywords['insult']=[]
# Creating a list of synonyms with multiple related words
new_list_syn = itertools.chain(list_syn['stupid'], list_syn['dumb'], list_syn['asshole'])
# Populating the values in the keywords dictionary with synonyms of keywords formatted with RegEx metacharacters 
for synonym in list(new_list_syn):
    keywords['insult'].append('.*\\b'+synonym+'\\b.*')

# Defining a new key in the keywords dictionary
keywords['suicide']=[]
# Creating a list of synonyms with multiple related words
new_list_syn = itertools.chain(list_syn['suicide'], list_syn['kill'], list_syn['die'])
# Populating the values in the keywords dictionary with synonyms of keywords formatted with RegEx metacharacters 
for synonym in list(new_list_syn):
    keywords['suicide'].append('.*\\b'+synonym+'\\b.*')

# Defining a new key in the keywords dictionary
keywords['automaticThought']=[]
# Creating a list of synonyms with multiple related words
new_list_syn = itertools.chain(list_syn['think'], list_syn['was'], list_syn['thought'], list_syn['thinking'])
# Populating the values in the keywords dictionary with synonyms of keywords formatted with RegEx metacharacters 
for synonym in list(new_list_syn):
    keywords['automaticThought'].append('.*\\b'+synonym+'\\b.*')

for intent, keys in keywords.items():
    # Joining the values in the keywords dictionary with the OR (|) operator updating them in keywords_dict dictionary
    keywords_dict[intent]=re.compile('|'.join(keys))
print (keywords_dict)

# Building a dictionary of responses
responses={
    'greetings':'Hello! So, tell me how have you been feeling?',
    'suicide':'If you are feeling vulnerable and need help, Please call 112 or 914 590 055 to seek help.',
    'fallback':'I dont quite understand. Could you repeat that?',
    'goodbyes':'Bye. Hope I\'ve been helpful.',
    'insult': 'Oh! that\'s mean :(',
    'automaticThought': 'When you were in that situtation and that thought came to you, how did you feel?'
}

def get_response(msg):
    matched_intent = None 
    for intent,pattern in keywords_dict.items():
        # Using the regular expression search function to look for keywords in user input
        if re.search(pattern, msg): 
            # if a keyword matches, select the corresponding intent from the keywords_dict dictionary
            matched_intent=intent  
    # The fallback intent is selected by default
    key='fallback' 
    if matched_intent in responses:
        # If a keyword matches, the fallback intent is replaced by the matched intent as the key for the responses dictionary
        key = matched_intent
        return responses[key]
    # The chatbot prints the response that matches the selected intent
    #print (responses[key])
    
    return 'none'


{'hello': {'hello', 'how do you do', 'hullo', 'howdy', 'hi'}, 'bye': {'adieu', 'bye bye', 'goodbye', 'good by', 'bye', 'au revoir', 'arrivederci', 'cheerio', 'auf wiedersehen', 'pass', 'adios', 'so long', 'sayonara', 'good bye', 'good day', 'goodby'}, 'suicide': {'self destruction', 'self annihilation', 'suicide', 'felo de se'}, 'kill': {'toss off', 'defeat', 'drink down', 'obliterate', 'killing', 'putting to death', 'wipe out', 'belt down', 'down', 'shoot down', 'vote out', 'kill', 'stamp out', 'pour down', 'bolt down', 'pop', 'vote down'}, 'die': {'perish', 'buy the farm', 'exit', 'pop off', 'cash in one s chips', 'conk', 'go', 'snuff it', 'go bad', 'become flat', 'drop dead', 'fail', 'give up the ghost', 'expire', 'kick the bucket', 'break', 'pall', 'die', 'dice', 'choke', 'break down', 'die out', 'conk out', 'give out', 'pass away', 'give way', 'decease', 'pass', 'croak'}, 'want': {'privation', 'need', 'deficiency', 'lack', 'desire', 'neediness', 'wishing', 'deprivation', 'require'

CHAT GUI

In [20]:
from tkinter import *
from nltk.sentiment import SentimentIntensityAnalyzer

BG_GRAY = "#ABB2B9"
BG_COLOR = "#17202A"
TEXT_COLOR = "#EAECEE"

FONT = "Helvetica 14"
FONT_BOLD = "Helvetica 13 bold"

bot_name = "Kai"
script_answers = [ 
              "I’m sorry that you feel this way. Did anything in particular trigger this feeling?\
               \nPlease provide a detailed explanation of the situation.",
              "What were you thinking at the time?",
              "Please, Rate how intense this feeling was (1-100)",
              "What has happened to make you believe the thought is true?",
              "What evidence is there that this thought is not true?",
              "Taking the information into account, is there an alternative way of thinking about the situation? Please give an example",
              "How would you rate your mood now? (1-100)",
             ]
CD = {
    'Always Being Right'            : 'DEFINITION: This way of thinking makes a person internalize their beliefs as facts and disregards the feelings of \
                                       the opposing participant in a conversation or debate.\
                                       \nLINK WITH TIPS: https://www.therapynowsf.com/blog/always-being-right-gearing-up-for-intellectual-battle',
    'Blaming'                       : 'DEFINITION: As a cognitive distortion, blaming involves attributing guilt or accountability for how we feel to another person.\
                                       \nLINK WITH TIPS: https://www.vacounseling.com/blaming-cognitive-distortion/',
    'Catastrophizing'               : 'DEFINITION: Predicting a negative outcome and jumping to the conclusion that if the negative outcome did in fact happen,\
                                       it would be a catastrophe.\
                                       \nLINK WITH TIPS: https://positivepsychology.com/catastrophizing/',
    'Control Fallacies'             : 'DEFINITION: The fallacy of control occurs when you believe you have too much power over a circumstance or your lot in life.\
                                       \nLINK WITH TIPS: https://www.therapynowsf.com/blog/control-fallacies-delving-deeper-into-cognitive-distortions',
    'Emotional Reasoning'           : 'DEFINITION: When a person uses emotional reasoning, a cognitive distortion, they come to the conclusion that something\
                                       is true regardless of the available evidence. Your thoughts are obscured by your emotions, and your reality\
                                       is similarly obscured.\
                                       \nLINK WITH TIPS: https://theeverygirl.com/17-of-the-easiest-ways-to-get-healthier-now/',
    'Fallacy Of Change'             : 'DEFINITION: This mental fallacy assumes that people ought to adapt to fit their own interests.\
                                       The person will exert pressure on others to alter their behavior because they believe doing so\
                                       will make them happier. They are convinced that changing a person is necessary for happiness.\
                                       \nLINK WITH TIPS: https://www.therapynowsf.com/blog/the-fallacy-of-change-and-the-pursuit-of-happiness',
    'Fallacy Of Fairness'           : 'DEFINITION: The idea that fairness and equality should be the foundation of everything in life is known as the Fallacy of Fairness.\
                                       \nhttps://www.therapynowsf.com/blog/the-fallacy-of-fairness-an-overview-of-this-cognitive-distortion',
    'Filtering'                     : 'DEFINITION: When we keep just the negative memories and the negative feelings associated with them, mental filtering takes place\
                                       and we reject the positive recollections.\
                                       \nLINK WITH TIPS: https://www.vacounseling.com/mental-filtering-cognitive-distortion/',
    'Global Labelling'              : 'DEFINITION: Labeling is a cognitive distortion in which we extrapolate from a single attribute of a person to that person as a whole.\
                                       \nLINK WITH TIPS: https://courageousandmindful.com/overcoming-labeling-and-mislabeling-a-cognitive-distortion/',
    'Heaven\'s Reward Fallacy'      : 'DEFINITION: The distortion known as Heaven\'s Reward Fallacy is founded on the justification that your rewards should be determined by how hard you work.\
                                       \nLINK WITH TIPS: https://cognitiontoday.com/heavens-reward-fallacy/',
    'Jumping to Conclusions'        : 'DEFINITION: Making arbitrary, negative assumptions about the thoughts of others or the future.\
                                       \nLINK WITH TIPS: https://www.therapynowsf.com/blog/jumping-to-conclusions-learn-how-to-stop-making-anxiety-fueled-mental-leaps',
    'Overgeneralization'            : 'DEFINITION: A type of cognitive distortion called overgeneralization occurs when someone extrapolates information from one incident to all other events.\
                                       \nLINK WITH TIPS: https://www.verywellmind.com/overgeneralization-3024614',
    'Personalization'               : 'DEFINITION: Personalization is the idea that something is totally your fault even when you had little to no influence on the outcome.\
                                       \nLINK WITH TIPS: https://www.therapynowsf.com/blog/personalization-a-common-type-of-negative-thinking',
    'Polarized'                     : 'DEFINITION: This distortion, also known as "all-or-nothing" or "black-and-white thinking,"\
                                       happens when people regularly think in extremes without taking all the relevant information into account.\
                                       \nLINK WITH TIPS: https://exploringyourmind.com/polarized-thinking-cognitive-distortion/',
    'Shoulds'                       : 'DEFINITION: "Should" statements are irrational rules you make for yourself and others without taking into account the particulars of a situation.\
                                       They are cognitive distortions. You tell yourself that there should be no exceptions to how things should be done.\
                                       \nLINK WITH TIPS: https://www.therapynowsf.com/blog/should-statements-reframe-the-way-you-think'
} 
             

class ChatGUI:
    
    def __init__(self):
        self.window = Tk()
        self._setup_main_window()
        self.msg_entry.delete(0, END)
        self._insert_kai_message("Hello!\
            \nMy name is Kai and I'm going to give you support to your self-directed CBT Therapy session.\
            \nSo, how have you been feeling?")
        # Steps that indicate in which stage the therapy session is
        self.step = 0
        self.sia = SentimentIntensityAnalyzer()
        self.automatic_thougth = None
        self.cognitive_distortion = None
        self.pre_mood_rating = None
        self.post_mood_rating = None

    def run(self):
        self.window.mainloop()
        
    def _setup_main_window(self):
        self.window.title("Chat")
        self.window.resizable(width=False, height=False)
        self.window.configure(width=470, height=550, bg=BG_COLOR)
        
        # head label
        head_label = Label(self.window, bg=BG_COLOR, fg=TEXT_COLOR,
                           text="Kai", font=FONT_BOLD, pady=10)
        head_label.place(relwidth=1)
        
        # tiny divider
        line = Label(self.window, width=450, bg=BG_GRAY)
        line.place(relwidth=1, rely=0.07, relheight=0.012)
        
        # text widget
        self.text_widget = Text(self.window, width=20, height=2, bg=BG_COLOR, fg=TEXT_COLOR,
                                font=FONT, padx=5, pady=5)
        self.text_widget.place(relheight=0.745, relwidth=1, rely=0.08)
        self.text_widget.configure(cursor="arrow", state=DISABLED)
        
        # scroll bar
        scrollbar = Scrollbar(self.text_widget)
        scrollbar.place(relheight=1, relx=0.974)
        scrollbar.configure(command=self.text_widget.yview)
        
        # bottom label
        bottom_label = Label(self.window, bg=BG_GRAY, height=80)
        bottom_label.place(relwidth=1, rely=0.825)
        
        # message entry box
        self.msg_entry = Entry(bottom_label, bg="#2C3E50", fg=TEXT_COLOR, font=FONT)
        self.msg_entry.place(relwidth=0.74, relheight=0.06, rely=0.008, relx=0.011)
        self.msg_entry.focus()
        self.msg_entry.bind("<Return>", self._on_enter_pressed)
        
        # send button
        send_button = Button(bottom_label, text="Send", font=FONT_BOLD, width=20, bg=BG_GRAY,
                             command=lambda: self._on_enter_pressed(None))
        send_button.place(relx=0.77, rely=0.008, relheight=0.06, relwidth=0.22)
    
    def _on_enter_pressed(self, event):
        msg = self.msg_entry.get()
        self._insert_message(msg, "You")
       
    def _insert_message(self, msg, sender):
        if not msg:
            return
        self.msg_entry.delete(0, END)
        msg1 = f"{sender}: {msg}\n\n"
        self.text_widget.configure(state=NORMAL)
        self.text_widget.insert(END, msg1)
        self.text_widget.configure(state=DISABLED)
        
        if (self.step == 3 or self.step == 7):
            self._insert_kai_message(self.check_rating(msg))
            if (self.step == 7 and (self.post_mood_rate - self.pre_mood_rate) >= 0):
                print("moood")
                self._insert_kai_message(self.relaxtion_exercise())

        else:
            kai_answer = get_response(msg)
            if (kai_answer == 'Hello! So, tell me how have you been feeling?' or 
                kai_answer == 'Bye. Hope I\'ve been helpful.'):
                self.step = 0
            if (kai_answer == 'none'):
                if (self.step == 0 and self.sia.polarity_scores(msg)["compound"] > 0):
                    self._insert_kai_message("I'm glad that you feel this way :)")
                    self.step = 0
                    
                else:
                    self._insert_kai_message(script_answers[self.step])
                    if (self.step == 1):
                        self.automatic_thought = msg
                        print(msg)
                    self.step = self.step + 1           
            else: 
                self._insert_kai_message(kai_answer)
                    
        self.text_widget.see(END)
          
    def _insert_kai_message(self, msg):
        msg1 = f"{bot_name}: {msg}\n\n"
        self.text_widget.configure(state=NORMAL)
        self.text_widget.insert(END, msg1)
        self.text_widget.configure(state=DISABLED)
    
    def check_rating(self, msg):
        kai_answer = "Please enter a valid answer"
        if (msg.isnumeric()):
                print('is numeric')
                rating = int(msg)
                if (rating >= 1 and rating <= 100):
                    if (self.step == 7):
                        kai_answer = self.detect_cognitive_distortion()
                        self.post_mood_rating = rating
                    else:
                        kai_answer = script_answers[self.step]
                        self.step += 1
                        self.pre_mood_rating = rating
        return kai_answer


    def detect_cognitive_distortion(self):
        print('HERE')
        pickled_model = pickle.load(open('cognitive_distortion_detector_model.pkl', 'rb'))
        self.cognitive_distortion = pickled_model.predict([self.automatic_thought])[0]
        print(self.cognitive_distortion)
        if (self.cognitive_distortion != "NO"):
            msg = "I have detected a potential cognitive distortion: " + self.cognitive_distortion + "\n" + CD[self.cognitive_distortion]
            return msg

    def relaxation_exercise():
        msg = "I have detected that your mood hasn't improved, so I recommend you to follow this link to do a relaxation exercise\
               in order to improve your mood" + "\n" + "LINK TO DEEP BREATHING EXERCISE VIDEO: https://www.youtube.com/watch?v=tEmt1Znux58"
        return msg
     
if __name__ == "__main__":
    app = ChatGUI()
    app.run()

Out of breath when I played in the park with my granddaughter

is numeric
is numeric
HERE
Blaming


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.2544.0_x64__qbz5n2kfra8p0\lib\tkinter\__init__.py", line 1921, in __call__
    return self.func(*args)
  File "C:\Users\Usuario\AppData\Local\Temp\ipykernel_19008\3577335431.py", line 127, in _on_enter_pressed
    self._insert_message(msg, "You")
  File "C:\Users\Usuario\AppData\Local\Temp\ipykernel_19008\3577335431.py", line 140, in _insert_message
    if (self.step == 7 and (self.post_mood_rate - self.pre_mood_rate) >= 0):
AttributeError: 'ChatGUI' object has no attribute 'post_mood_rate'
